In [25]:
import numpy as np
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import matplotlib.pyplot as plt
import matplotlib.ticker
import math
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(123)

# Random number generators in other libraries
np.random.seed(123)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': print(torch.cuda.get_device_name()) 

cpu


# Data Prep

Training and Testing data is prepared from the solution file

In [26]:
n = 32
x_1 = np.linspace(-1, 1, 2*n+1)
y_1 = np.linspace(1, 0, n+1)
z_1 = np.linspace(1, -1, 2*n+1)
Z_1, X_1, Y_1 = np.meshgrid(z_1, x_1, y_1)
x_2 = np.linspace(-1, 0, n+1)
y_2 = np.linspace(-1/n, -1, n)
Z_2, X_2, Y_2 = np.meshgrid(z_1, x_2, y_2)

x = np.vstack((X_1.flatten(order='F')[:, None], X_2.flatten(order='F')[:, None]))
y = np.vstack((Y_1.flatten(order='F')[:, None], Y_2.flatten(order='F')[:, None]))
z = np.vstack((Z_1.flatten(order='F')[:, None], Z_2.flatten(order='F')[:, None]))

# Test Data

We prepare the test data to compare against the solution produced by the PINN.

In [27]:
X_v_test = np.hstack((x, y, z))
v_true = (x-x**3)*(y-y**3)*(1-z**2)
r = np.sqrt(x**2 + y**2)
theta = np.arctan2(y, x)
theta = np.where(theta < 0, theta + 2*np.pi, theta)
sd = r**(2/3) * np.sin(2*theta/3)
R = 1/2
etad = np.where(r < R, 15 / 16 * (
                    8 / 15 - (4 * r / R - 3) + 2 / 3 * (4 * r / R - 3) ** 3 - 1 / 5 * (4 * r / R - 3) ** 5), 0)
etad = np.where(r < R/2, 1, etad)

PHI = 2 * np.arctan(-math.e**(-np.pi*r)*np.sin(np.pi*z)/(1+math.e**(-np.pi*r)*np.cos(np.pi*z)))
u_true = v_true + sd * etad * PHI

lb = np.array([-1, -1, -1]) #lower bound
ub = np.array([1, 1, 1])  #upper bound

C:\Users\HP\AppData\Local\Temp/ipykernel_7328/2403138763.py:12: RuntimeWarning: divide by zero encountered in true_divide
  PHI = 2 * np.arctan(-math.e**(-np.pi*r)*np.sin(np.pi*z)/(1+math.e**(-np.pi*r)*np.cos(np.pi*z)))


# Training Data

In [28]:
def trainingdata(N_v,N_f):
    
    x_1 = np.linspace(-1,1,2*n+1)
    x_3 = np.linspace((n-1)/n,-(n-1)/n,2*n-1)
    X, Z = np.meshgrid(x_1,x_3)
    edge1_x = np.hstack((X.flatten('F')[:,None], np.linspace(1,1,(2*n+1)*(2*n-1))[:,None], Z.flatten('F')[:,None]))
    edge1_v = np.linspace(0,0,(2*n+1)*(2*n-1))[:,None]
    
    x_3 = np.linspace((n-1)/n,-(n-1)/n,2*n-1)
    x_2 = np.linspace(0,(n-1)/n,n)
    Y, Z = np.meshgrid(x_2,x_3)
    edge2_x = np.hstack((np.linspace(1,1,n*(2*n-1))[:,None], Y.flatten('F')[:,None], Z.flatten('F')[:,None]))
    edge2_v = np.linspace(0,0,n*(2*n+1))[:,None]
    
    x_3 = np.linspace((n-1)/n,-(n-1)/n,2*n-1)
    x_1 = np.linspace(0,(n-1)/n,n)
    X, Z = np.meshgrid(x_1,x_3)
    edge3_x = np.hstack((X.flatten('F')[:,None], np.linspace(0,0,n*(2*n-1))[:,None], Z.flatten('F')[:,None]))
    edge3_v = np.linspace(0,0,n*(2*n-1))[:,None]
    
    x_3 = np.linspace((n-1)/n,-(n-1)/n,2*n-1)
    x_2 = np.linspace(-1,-1/n,n)
    Y, Z = np.meshgrid(x_2,x_3)
    edge4_x = np.hstack((np.linspace(0,0,n*(2*n-1))[:,None], Y.flatten('F')[:,None], Z.flatten('F')[:,None]))
    edge4_v = np.linspace(0,0,n*(2*n-1))[:,None]
    
    x_3 = np.linspace((n-1)/n,-(n-1)/n,2*n-1)
    x_1 = np.linspace(-1,-1/n,n)
    X, Z = np.meshgrid(x_1,x_3)
    edge5_x = np.hstack((X.flatten('F')[:,None], np.linspace(-1,-1,n*(2*n-1))[:,None], Z.flatten('F')[:,None]))
    edge5_v = np.linspace(0,0,n*(2*n-1))[:,None]
    
    x_3 = np.linspace((n-1)/n,-(n-1)/n,2*n-1)
    x_2 = np.linspace(-(n-1)/n,(n-1)/n,2*n-1)
    Y, Z = np.meshgrid(x_2,x_3)
    edge6_x = np.hstack((np.linspace(-1,-1,(2*n-1)**2)[:,None], Y.flatten('F')[:,None], Z.flatten('F')[:,None]))
    edge6_v = np.linspace(0,0,(2*n-1)**2)[:,None]
    
    x_1 = np.linspace(-1, 1, 2*n+1)
    y_1 = np.linspace(1, 0, n+1)
    Y_1, X_1 = np.meshgrid(y_1, x_1)
    x_2 = np.linspace(-1, 0, n+1)
    y_2 = np.linspace(-1/n, -1, n)
    Y_2, X_2 = np.meshgrid(y_2, x_2)
    x = np.vstack((X_1.flatten(order='F')[:, None], X_2.flatten(order='F')[:, None]))
    y = np.vstack((Y_1.flatten(order='F')[:, None], Y_2.flatten(order='F')[:, None]))
    
    edge7_x = np.hstack((x, y, np.linspace(-1,-1,(3*n+1)*(n+1))[:,None]))
    edge7_v = np.linspace(0,0,(3*n+1)*(n+1))[:,None]
    edge8_x = np.hstack((x, y, np.linspace(1,1,(3*n+1)*(n+1))[:,None]))
    edge8_v = edge7_v
    
    all_X_v_train = np.vstack([edge1_x, edge2_x, edge3_x, edge4_x, edge5_x, edge6_x, edge7_x, edge8_x])
    all_v_train = np.vstack([edge1_v, edge2_v, edge3_v, edge4_v, edge5_v, edge6_v, edge7_v, edge8_v])

    # choose random N_v points for training
    all_X_v_train_r = np.sqrt(all_X_v_train[:, 0]**2 + all_X_v_train[:, 1]**2)
    probability = np.where(all_X_v_train_r == 0, 0, 1)
    probability = probability / np.sum(probability)
    idx = np.random.choice(all_X_v_train.shape[0], N_v, replace=False, p=probability)
    X_v_train = all_X_v_train[idx[0:N_v], :]  # choose indices from  set 'idx' (x,t)
    v_train = all_v_train[idx[0:N_v], :]
    
    '''Collocation Points'''

    # N_f sets of tuples(x,t)
#     probability = np.where(r == 0, 0, 1)
#     probability = probability / np.sum(probability)
#     idx = np.random.choice(X_v_test.shape[0], N_f, replace=False, p=probability.T[0])
#     X_f = X_v_test[idx[0:N_f], :]

    prob = np.where(np.random.rand(1, N_f) < 2/3, 1, 0)
    N = np.sum(prob)
    print(N)
    
    lb = np.array([-1, -1, -1])
    ub = np.array([0, 1, 1])
    X_f_1 = lb + (ub - lb) * lhs(3, N)
    lb = np.array([0, 0, -1])
    ub = np.array([1, 1, 1])
    X_f_2 = lb + (ub - lb) * lhs(3, N_f - N)
    X_f = np.vstack((X_f_1, X_f_2))

    X_f_train = np.vstack((X_f, X_v_train))  # append training points to collocation points

    return X_f_train, X_v_train, v_train

# PINN

Creating sequential layers using the class
tf.Module

In [29]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
    
        'Initialise neural network as a nn.MSELosslist using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        self.lambdah = torch.autograd.Variable(torch.ones(21).to(device),requires_grad=True)
        
        'Xavier Normal Initialization'
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            
            # weights from a normal distribution with 
            # Recommended gain value for tanh = 5/3?
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
            
    def forward(self,x):
        
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            
            z = self.linears[i](a)
                        
            a = self.activation(z)
            
        a = self.linears[-1](a)
        
        return a
                        
    def loss_BC(self,x,y):
        
        loss_v = self.loss_function(self.forward(x), y)
                
        return loss_v
    
    def loss_PDE(self, x_to_train_f):
                
        x_1_f = x_to_train_f[:,[0]]
        x_2_f = x_to_train_f[:,[1]]
        x_3_f = x_to_train_f[:,[2]]
                        
        g = x_to_train_f.clone()
                        
        g.requires_grad = True
        
        vv = self.forward(g)
                
        v_x = autograd.grad(vv,g,torch.ones([x_to_train_f.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]        
        v_xx = autograd.grad(v_x[:,[0]],g,torch.ones([x_to_train_f.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
        v_yy = autograd.grad(v_x[:,[1]],g,torch.ones([x_to_train_f.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
        v_zz = autograd.grad(v_x[:,[2]],g,torch.ones([x_to_train_f.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0]
        
        v_xx_1 = v_xx[:,[0]]
        v_xx_2 = v_yy[:,[1]]
        v_xx_3 = v_zz[:,[2]]
                        
        r = torch.sqrt(x_1_f ** 2 + x_2_f ** 2)
        theta = np.arctan2(x_2_f, x_1_f)
        theta = torch.where(theta < 0, theta + 2 * np.pi, theta)
        z = x_3_f
        
        deltap = torch.where(r < R, (2*r**(2/3)*np.sin((2*theta)/3)*((2*np.pi**2*math.e**(-3*np.pi*r)*np.sin(np.pi*z)**3)/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**3 - (np.pi**2*math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1) + (3*np.pi**2*math.e**(-2*np.pi*r)*np.cos(np.pi*z)*np.sin(np.pi*z))/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2)*((15*r)/2 - (5*(8*r - 3)**3)/8 + (3*(8*r - 3)**5)/16 - 53/16))/((math.e**(-2*np.pi*r)*np.sin(np.pi*z)**2)/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2 + 1) - (8*np.arctan((math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1))*np.sin((2*theta)/3)*((15*r)/2 - (5*(8*r - 3)**3)/8 + (3*(8*r - 3)**5)/16 - 53/16))/(9*r**(4/3)) - (r*((4*np.arctan((math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1))*np.sin((2*theta)/3)*((15*r)/2 - (5*(8*r - 3)**3)/8 + (3*(8*r - 3)**5)/16 - 53/16))/(9*r**(4/3)) - 2*r**(2/3)*np.arctan((math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1))*np.sin((2*theta)/3)*(240*(8*r - 3)**3 - 1920*r + 720) - (8*np.arctan((math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1))*np.sin((2*theta)/3)*((15*(8*r - 3)**4)/2 - 15*(8*r - 3)**2 + 15/2))/(3*r**(1/3)) + (8*np.sin((2*theta)/3)*((np.pi*math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1) - (np.pi*math.e**(-2*np.pi*r)*np.cos(np.pi*z)*np.sin(np.pi*z))/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2)*((15*r)/2 - (5*(8*r - 3)**3)/8 + (3*(8*r - 3)**5)/16 - 53/16))/(3*r**(1/3)*((math.e**(-2*np.pi*r)*np.sin(np.pi*z)**2)/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2 + 1)) - (2*r**(2/3)*np.sin((2*theta)/3)*((np.pi**2*math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1) + (2*np.pi**2*math.e**(-3*np.pi*r)*np.cos(np.pi*z)**2*np.sin(np.pi*z))/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**3 - (3*np.pi**2*math.e**(-2*np.pi*r)*np.cos(np.pi*z)*np.sin(np.pi*z))/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2)*((15*r)/2 - (5*(8*r - 3)**3)/8 + (3*(8*r - 3)**5)/16 - 53/16))/((math.e**(-2*np.pi*r)*np.sin(np.pi*z)**2)/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2 + 1) + (4*r**(2/3)*np.sin((2*theta)/3)*((np.pi*math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1) - (np.pi*math.e**(-2*np.pi*r)*np.cos(np.pi*z)*np.sin(np.pi*z))/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2)*((15*(8*r - 3)**4)/2 - 15*(8*r - 3)**2 + 15/2))/((math.e**(-2*np.pi*r)*np.sin(np.pi*z)**2)/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2 + 1) + (2*r**(2/3)*np.sin((2*theta)/3)*((2*np.pi*math.e**(-2*np.pi*r)*np.sin(np.pi*z)**2)/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2 - (2*np.pi*math.e**(-3*np.pi*r)*np.cos(np.pi*z)*np.sin(np.pi*z)**2)/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**3)*((np.pi*math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1) - (np.pi*math.e**(-2*np.pi*r)*np.cos(np.pi*z)*np.sin(np.pi*z))/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2)*((15*r)/2 - (5*(8*r - 3)**3)/8 + (3*(8*r - 3)**5)/16 - 53/16))/((math.e**(-2*r*np.pi)*np.sin(z*np.pi)**2)/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1)**2 + 1)**2) - (4*np.arctan((math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1))*np.sin((2*theta)/3)*((15*r)/2 - (5*(8*r - 3)**3)/8 + (3*(8*r - 3)**5)/16 - 53/16))/(3*r**(1/3)) - 2*r**(2/3)*np.arctan((math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1))*np.sin((2*theta)/3)*((15*(8*r - 3)**4)/2 - 15*(8*r - 3)**2 + 15/2) + (2*r**(2/3)*np.sin((2*theta)/3)*((np.pi*math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1) - (np.pi*math.e**(-2*np.pi*r)*np.cos(np.pi*z)*np.sin(np.pi*z))/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2)*((15*r)/2 - (5*(8*r - 3)**3)/8 + (3*(8*r - 3)**5)/16 - 53/16))/((math.e**(-2*np.pi*r)*np.sin(np.pi*z)**2)/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2 + 1))/r - (2*r**(2/3)*np.sin((2*theta)/3)*((2*np.pi*math.e**(-3*np.pi*r)*np.sin(np.pi*z)**3)/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**3 + (2*np.pi*math.e**(-2*np.pi*r)*np.cos(np.pi*z)*np.sin(np.pi*z))/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2)*((np.pi*math.e**(-2*np.pi*r)*np.sin(np.pi*z)**2)/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2 + (np.pi*math.e**(-np.pi*r)*np.cos(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1))*((15*r)/2 - (5*(8*r - 3)**3)/8 + (3*(8*r - 3)**5)/16 - 53/16))/((math.e**(-2*r*np.pi)*np.sin(z*np.pi)**2)/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1)**2 + 1)**2, 0)
        deltap = torch.where(r < R / 2, (r*((4*np.arctan((math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1))*np.sin((2*theta)/3))/(9*r**(4/3)) + (8*np.sin((2*theta)/3)*((np.pi*math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1) - (np.pi*math.e**(-2*np.pi*r)*np.cos(np.pi*z)*np.sin(np.pi*z))/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2))/(3*r**(1/3)*((math.e**(-2*np.pi*r)*np.sin(np.pi*z)**2)/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2 + 1)) - (2*r**(2/3)*np.sin((2*theta)/3)*((np.pi**2*math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1) + (2*np.pi**2*math.e**(-3*np.pi*r)*np.cos(np.pi*z)**2*np.sin(np.pi*z))/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**3 - (3*np.pi**2*math.e**(-2*np.pi*r)*np.cos(np.pi*z)*np.sin(np.pi*z))/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2))/((math.e**(-2*np.pi*r)*np.sin(np.pi*z)**2)/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2 + 1) + (2*r**(2/3)*np.sin((2*theta)/3)*((2*np.pi*math.e**(-2*np.pi*r)*np.sin(np.pi*z)**2)/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2 - (2*np.pi*math.e**(-3*np.pi*r)*np.cos(np.pi*z)*np.sin(np.pi*z)**2)/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**3)*((np.pi*math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1) - (np.pi*math.e**(-2*np.pi*r)*np.cos(np.pi*z)*np.sin(np.pi*z))/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2))/((math.e**(-2*r*np.pi)*np.sin(z*np.pi)**2)/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1)**2 + 1)**2) - (4*np.arctan((math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1))*np.sin((2*theta)/3))/(3*r**(1/3)) + (2*r**(2/3)*np.sin((2*theta)/3)*((np.pi*math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1) - (np.pi*math.e**(-2*np.pi*r)*np.cos(np.pi*z)*np.sin(np.pi*z))/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2))/((math.e**(-2*np.pi*r)*np.sin(np.pi*z)**2)/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2 + 1))/r + (8*np.arctan((math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1))*np.sin((2*theta)/3))/(9*r**(4/3)) - (2*r**(2/3)*np.sin((2*theta)/3)*((2*np.pi**2*math.e**(-3*np.pi*r)*np.sin(np.pi*z)**3)/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**3 - (np.pi**2*math.e**(-np.pi*r)*np.sin(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1) + (3*np.pi**2*math.e**(-2*np.pi*r)*np.cos(np.pi*z)*np.sin(np.pi*z))/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2))/((math.e**(-2*np.pi*r)*np.sin(np.pi*z)**2)/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2 + 1) + (2*r**(2/3)*np.sin((2*theta)/3)*((2*np.pi*math.e**(-3*np.pi*r)*np.sin(np.pi*z)**3)/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**3 + (2*np.pi*math.e**(-2*np.pi*r)*np.cos(np.pi*z)*np.sin(np.pi*z))/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2)*((np.pi*math.e**(-2*np.pi*r)*np.sin(np.pi*z)**2)/(math.e**(-r*np.pi)*np.cos(z*np.pi) + 1)**2 + (np.pi*math.e**(-np.pi*r)*np.cos(np.pi*z))/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1)))/((math.e**(-2*r*np.pi)*np.sin(z*np.pi)**2)/(math.e**(-np.pi*r)*np.cos(np.pi*z) + 1)**2 + 1)**2, deltap)
        
        f = 6 * x_1_f * (x_2_f - x_2_f**3) * (1 - x_3_f**2) + 6 * x_2_f * (x_1_f - x_1_f**3) * (1 - x_3_f**2) + 2 * (x_2_f - x_2_f**3) * (x_1_f - x_1_f**3) - deltap
        
        deltap0 = torch.where(r < R,
                          -4 * (-7.5 * r - 0.1875 * (8 * r - 3) ** 5 + 0.625 * (8 * r - 3) ** 3 + 3.3125) * np.sin(
                              2 * theta / 3) / (9 * r ** (4 / 3)) + (2 / 3 * (
                                  -7.5 * r - 0.1875 * (8 * r - 3) ** 5 + 0.625 * (
                                  8 * r - 3) ** 3 + 3.3125) * np.sin(2 * theta / 3) / r ** (1 / 3) + r ** (
                                                                             2 / 3) * (
                                                                             -7.5 * (8 * r - 3) ** 4 + 15.0 * (
                                                                             8 * r - 3) ** 2 - 7.5) * np.sin(
                              2 * theta / 3) + r * (-2 / 9 * (-7.5 * r - 0.1875 * (8 * r - 3) ** 5 + 0.625 * (
                                  8 * r - 3) ** 3 + 3.3125) * np.sin(2 * theta / 3) / r ** (4 / 3) + 4 / 3 * (
                                                            -7.5 * (8 * r - 3) ** 4 + 15.0 * (
                                                            8 * r - 3) ** 2 - 7.5) * np.sin(
                              2 * theta / 3) / r ** (1 / 3) + r ** (2 / 3) * (
                                                            1920.0 * r - 240.0 * (8 * r - 3) ** 3 - 720.0) * np.sin(
                              2 * theta / 3))) / r, 0)
        deltap0 = torch.where(r < R / 2, 0, deltap0)
        
        deltaphat = self.lambdah[0] * deltap0
        for n in range(1,21):
            phat = torch.where(r < R, (r*(9.86960440108936*n**2*r**0.666666666666667*(-7.5*r - 0.1875*(8*r - 3)**5 + 0.625*(8*r - 3)**3 + 3.3125)*np.sin(2*theta/3)*np.sin(3.14159265358979*n*z)/2.71828182845905**(3.14159265358979*n*r) - 4.18879020478639*n*(-7.5*r - 0.1875*(8*r - 3)**5 + 0.625*(8*r - 3)**3 + 3.3125)*np.sin(2*theta/3)*np.sin(3.14159265358979*n*z)/(2.71828182845905**(3.14159265358979*n*r)*r**0.333333333333333) - 6.28318530717959*n*r**0.666666666666667*(-7.5*(8*r - 3)**4 + 15.0*(8*r - 3)**2 - 7.5)*np.sin(2*theta/3)*np.sin(3.14159265358979*n*z)/2.71828182845905**(3.14159265358979*n*r) - 0.222222222222222*(-7.5*r - 0.1875*(8*r - 3)**5 + 0.625*(8*r - 3)**3 + 3.3125)*np.sin(2*theta/3)*np.sin(3.14159265358979*n*z)/(2.71828182845905**(3.14159265358979*n*r)*r**1.33333333333333) + 1.33333333333333*(-7.5*(8*r - 3)**4 + 15.0*(8*r - 3)**2 - 7.5)*np.sin(2*theta/3)*np.sin(3.14159265358979*n*z)/(2.71828182845905**(3.14159265358979*n*r)*r**0.333333333333333) + r**0.666666666666667*(1920.0*r - 240.0*(8*r - 3)**3 - 720.0)*np.sin(2*theta/3)*np.sin(3.14159265358979*n*z)/2.71828182845905**(3.14159265358979*n*r)) - 3.14159265358979*n*r**0.666666666666667*(-7.5*r - 0.1875*(8*r - 3)**5 + 0.625*(8*r - 3)**3 + 3.3125)*np.sin(2*theta/3)*np.sin(3.14159265358979*n*z)/2.71828182845905**(3.14159265358979*n*r) + 0.666666666666667*(-7.5*r - 0.1875*(8*r - 3)**5 + 0.625*(8*r - 3)**3 + 3.3125)*np.sin(2*theta/3)*np.sin(3.14159265358979*n*z)/(2.71828182845905**(3.14159265358979*n*r)*r**0.333333333333333) + r**0.666666666666667*(-7.5*(8*r - 3)**4 + 15.0*(8*r - 3)**2 - 7.5)*np.sin(2*theta/3)*np.sin(3.14159265358979*n*z)/2.71828182845905**(3.14159265358979*n*r))/r - 9.86960440108936*n**2*r**0.666666666666667*(-7.5*r - 0.1875*(8*r - 3)**5 + 0.625*(8*r - 3)**3 + 3.3125)*np.sin(2*theta/3)*np.sin(3.14159265358979*n*z)/2.71828182845905**(3.14159265358979*n*r) - 4*(-7.5*r - 0.1875*(8*r - 3)**5 + 0.625*(8*r - 3)**3 + 3.3125)*np.sin(2*theta/3)*np.sin(3.14159265358979*n*z)/(9*2.71828182845905**(3.14159265358979*n*r)*r**1.33333333333333), 0)
            phat = torch.where(r < R / 2, (r*(9.86960440108936*n**2*r**0.666666666666667*np.sin(2*theta/3)*np.sin(3.14159265358979*n*z)/2.71828182845905**(3.14159265358979*n*r) - 4.18879020478639*n*np.sin(2*theta/3)*np.sin(3.14159265358979*n*z)/(2.71828182845905**(3.14159265358979*n*r)*r**0.333333333333333) - 0.222222222222222*np.sin(2*theta/3)*np.sin(3.14159265358979*n*z)/(2.71828182845905**(3.14159265358979*n*r)*r**1.33333333333333)) - 3.14159265358979*n*r**0.666666666666667*np.sin(2*theta/3)*np.sin(3.14159265358979*n*z)/2.71828182845905**(3.14159265358979*n*r) + 0.666666666666667*np.sin(2*theta/3)*np.sin(3.14159265358979*n*z)/(2.71828182845905**(3.14159265358979*n*r)*r**0.333333333333333))/r - 9.86960440108936*n**2*r**0.666666666666667*np.sin(2*theta/3)*np.sin(3.14159265358979*n*z)/2.71828182845905**(3.14159265358979*n*r) - 4*np.sin(2*theta/3)*np.sin(3.14159265358979*n*z)/(9*2.71828182845905**(3.14159265358979*n*r)*r**1.33333333333333), phat)
            deltaphat = deltaphat + self.lambdah[n]*phat
        
        F = v_xx_1 + v_xx_2 + v_xx_3 + f + deltaphat
        
        loss_f = self.loss_function(F, f_hat)
        
        return loss_f
    
    def loss(self,x,y,x_to_train_f,sigma):

        loss_v = self.loss_BC(x,y)
        loss_f = self.loss_PDE(x_to_train_f)

        loss = sigma * loss_v + loss_f

        return loss
     
    'callable for optimizer'                                       
    def closure(self):
        
        optimizer.zero_grad()
        
        loss_val = self.loss(X_v_train, v_train, X_f_train, sigma)
        
        #error_vec, _ = PINN.test()
        
        #print(loss_val,error_vec)
        
        global ite, iteration_vec, error_vec, fun_vec
        ite = ite + 1
        
        if (ite % 100 == 0):
            iteration_vec.append(ite)
            fun_vec.append(loss_val.item())
            error, _ = self.test()
            error_vec.append(error.item())
        
        loss_val.backward()

        return loss_val        
    
    def test(self):
                
        v_pred = self.forward(X_v_test_tensor)
        
        error_vec = torch.linalg.norm((v-v_pred),2)/torch.linalg.norm(v,2)        # Relative L2 Norm of the error (Vector)
        
        u_pred = v_pred.cpu().detach().numpy()
        
        u_pred = u_pred + self.lambdah[0].cpu().detach().numpy() * sd * etad
        
        for n in range(1,21):
            PHInd = math.e**(-n*np.pi*r)*np.sin(n*np.pi*z)
            u_pred = u_pred + self.lambdah[n].cpu().detach().numpy() * sd * etad * PHInd
            
        print('lambdah:',self.lambdah)
        
        return error_vec, u_pred

# Loss Function

The loss function consists of two parts:

    loss_BC: MSE error of boundary losses
    loss_PDE: MSE error of collocation points satisfying the PDE

loss = loss_BC + loss_PDE


In [30]:
N_v = 800
N_f = 10000

X_f_train_np_array, X_v_train_np_array, v_train_np_array = trainingdata(N_v,N_f)

'Convert to tensor and send to GPU'
X_f_train = torch.from_numpy(X_f_train_np_array).float().to(device)
X_v_train = torch.from_numpy(X_v_train_np_array).float().to(device)
#v_train = torch.zeros(X_v_train.shape[0],1).to(device)
v_train = torch.from_numpy(v_train_np_array).float().to(device)
X_v_test_tensor = torch.from_numpy(X_v_test).float().to(device)
v = torch.from_numpy(v_true).float().to(device)
u = torch.from_numpy(u_true).float().to(device)
f_hat = torch.zeros(X_f_train.shape[0],1).to(device)

iteration_vec = []
fun_vec = []
error_vec = []
lambda0_vec = []
lambda1_vec = []
lambda2_vec = []
lambda3_vec = []
lambda4_vec = []
lambda5_vec = []

layers = np.array([3,10,10,10,1])

PINN = Sequentialmodel(layers)
       
PINN.to(device)

'Neural Network Summary'

print(PINN)
params = list(PINN.parameters())

sigma = 400

start_time = time.time()
error_0 = 1

optimizer = optim.Adam(PINN.parameters(), lr=1e-3,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
optimizerl = optim.Adam([PINN.lambdah], lr=6e-3,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
#optimizer = optim.Adam([PINN.lambdah], lr=0.0001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
max_iter = 2000
for i in range(max_iter):
    loss = PINN.loss(X_v_train, v_train, X_f_train, sigma)
    optimizer.zero_grad()     # zeroes the gradient buffers of all parameters
    optimizerl.zero_grad()     # zeroes the gradient buffers of all parameters
    loss.backward()       #backprop
    optimizer.step()
    optimizerl.step()
    if i%100==0:
        lambda0_vec.append(PINN.lambdah[0].item())
        lambda1_vec.append(PINN.lambdah[1].item())
        lambda2_vec.append(PINN.lambdah[2].item())
        lambda3_vec.append(PINN.lambdah[3].item())
        lambda4_vec.append(PINN.lambdah[4].item())
        lambda5_vec.append(PINN.lambdah[5].item())
        if i%200==0:
            iteration_vec.append(i)
            fun_vec.append(loss.item())
            error, _ = PINN.test()
            error_vec.append(error.item())

ite = 2000
while sigma<4000 and error_0>0.01:
    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.1,
                              max_iter = 2500, 
                              max_eval = None, 
                              tolerance_grad = 1e-06, 
                              tolerance_change = 1e-09, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')

    optimizer.zero_grad()     # zeroes the gradient buffers of all parameters
    optimizer.step(PINN.closure)
    sigma = 1.5 * sigma
    error_0, _ = PINN.test()
    print(error_0)

elapsed = time.time() - start_time                
print('Training time: %.2f' % (elapsed))

''' Model Accuracy ''' 
error_0, u_pred = PINN.test()

print('sigma: %f' %(sigma/1.5))
print('Test Error: %.5f'  % (error_0))
print('iteration: %d' %(ite))

print(iteration_vec,fun_vec,error_vec)
print(lambda0_vec,'\n',lambda1_vec,'\n',lambda2_vec,'\n',lambda3_vec,'\n',lambda4_vec,'\n',lambda5_vec)

6694
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=1, bias=True)
  )
)
lambdah: tensor([0.9940, 0.9940, 1.0060, 0.9940, 0.9940, 0.9940, 0.9940, 0.9940, 0.9940,
        0.9940, 1.0060, 1.0060, 1.0060, 1.0060, 0.9940, 0.9940, 0.9940, 0.9940,
        0.9940, 0.9940, 1.0060], requires_grad=True)
lambdah: tensor([ 0.1732, -0.1065,  1.0457,  0.0603,  0.4501,  0.0752,  0.4153, -0.0071,
         0.2466, -0.0124,  0.8447,  0.6085,  0.6397,  0.7943,  0.7265,  0.2363,
         0.4426,  0.4249,  0.6603,  0.8095,  1.7298], requires_grad=True)
lambdah: tensor([ 0.0223, -0.9362,  1.0737, -0.4041,  0.5337, -0.3008,  0.2528, -0.2999,
         0.0331, -0.4052,  0.5429,  0.2782,  0.3899,  0.4150,  0.6117,  0.0132,
         0.2782,  0.234

lambdah: tensor([-7.7930e-05, -1.9999e+00,  1.0063e+00, -6.5775e-01,  5.1352e-01,
        -4.2379e-01,  3.2705e-01, -3.6189e-01,  2.4103e-01, -2.3835e-01,
         2.7236e-01, -1.8545e-01,  3.6965e-01, -1.0452e-02,  2.7613e-01,
        -2.0816e-01,  2.8666e-01,  1.2388e-03,  1.9607e-01,  7.7738e-02,
         4.9549e-01], requires_grad=True)
lambdah: tensor([-7.7930e-05, -1.9999e+00,  1.0063e+00, -6.5775e-01,  5.1352e-01,
        -4.2379e-01,  3.2705e-01, -3.6189e-01,  2.4103e-01, -2.3835e-01,
         2.7236e-01, -1.8545e-01,  3.6965e-01, -1.0452e-02,  2.7613e-01,
        -2.0816e-01,  2.8666e-01,  1.2388e-03,  1.9607e-01,  7.7738e-02,
         4.9549e-01], requires_grad=True)
lambdah: tensor([-7.7930e-05, -1.9999e+00,  1.0063e+00, -6.5775e-01,  5.1352e-01,
        -4.2379e-01,  3.2705e-01, -3.6189e-01,  2.4103e-01, -2.3835e-01,
         2.7236e-01, -1.8545e-01,  3.6965e-01, -1.0452e-02,  2.7613e-01,
        -2.0816e-01,  2.8666e-01,  1.2388e-03,  1.9607e-01,  7.7738e-02,
         4.95

lambdah: tensor([-7.7930e-05, -1.9999e+00,  1.0063e+00, -6.5775e-01,  5.1352e-01,
        -4.2379e-01,  3.2705e-01, -3.6189e-01,  2.4103e-01, -2.3835e-01,
         2.7236e-01, -1.8545e-01,  3.6965e-01, -1.0452e-02,  2.7613e-01,
        -2.0816e-01,  2.8666e-01,  1.2388e-03,  1.9607e-01,  7.7738e-02,
         4.9549e-01], requires_grad=True)
lambdah: tensor([-7.7930e-05, -1.9999e+00,  1.0063e+00, -6.5775e-01,  5.1352e-01,
        -4.2379e-01,  3.2705e-01, -3.6189e-01,  2.4103e-01, -2.3835e-01,
         2.7236e-01, -1.8545e-01,  3.6965e-01, -1.0452e-02,  2.7613e-01,
        -2.0816e-01,  2.8666e-01,  1.2388e-03,  1.9607e-01,  7.7738e-02,
         4.9549e-01], requires_grad=True)
lambdah: tensor([-7.7930e-05, -1.9999e+00,  1.0063e+00, -6.5775e-01,  5.1352e-01,
        -4.2379e-01,  3.2705e-01, -3.6189e-01,  2.4103e-01, -2.3835e-01,
         2.7236e-01, -1.8545e-01,  3.6965e-01, -1.0452e-02,  2.7613e-01,
        -2.0816e-01,  2.8666e-01,  1.2388e-03,  1.9607e-01,  7.7738e-02,
         4.95

lambdah: tensor([-7.7930e-05, -1.9999e+00,  1.0063e+00, -6.5775e-01,  5.1352e-01,
        -4.2379e-01,  3.2705e-01, -3.6189e-01,  2.4103e-01, -2.3835e-01,
         2.7236e-01, -1.8545e-01,  3.6965e-01, -1.0452e-02,  2.7613e-01,
        -2.0816e-01,  2.8666e-01,  1.2388e-03,  1.9607e-01,  7.7738e-02,
         4.9549e-01], requires_grad=True)
tensor(0.0505, grad_fn=<DivBackward0>)
lambdah: tensor([-7.7930e-05, -1.9999e+00,  1.0063e+00, -6.5775e-01,  5.1352e-01,
        -4.2379e-01,  3.2705e-01, -3.6189e-01,  2.4103e-01, -2.3835e-01,
         2.7236e-01, -1.8545e-01,  3.6965e-01, -1.0452e-02,  2.7613e-01,
        -2.0816e-01,  2.8666e-01,  1.2388e-03,  1.9607e-01,  7.7738e-02,
         4.9549e-01], requires_grad=True)
tensor(0.0505, grad_fn=<DivBackward0>)
lambdah: tensor([-7.7930e-05, -1.9999e+00,  1.0063e+00, -6.5775e-01,  5.1352e-01,
        -4.2379e-01,  3.2705e-01, -3.6189e-01,  2.4103e-01, -2.3835e-01,
         2.7236e-01, -1.8545e-01,  3.6965e-01, -1.0452e-02,  2.7613e-01,
        